In [ ]:
import sys
import os

# Go up two levels from notebook (Training/MLR) to project root
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_root)

print("Project root added to sys.path:", project_root)

# Ensure the model save directory exists
model_save_path = os.path.join('.')
os.makedirs(model_save_path, exist_ok=True)  # Creates directory if it doesn't exist

In [ ]:
import pandas as pd
from sklearn import linear_model
from Training.Helper.dataPreprocessing import TRAIN_DATA_PATH_1990S, integer_index

date_col = 'observation_date'

# Load and format training data (only using PCEPI)
train_df = pd.read_csv(TRAIN_DATA_PATH_1990S, parse_dates=[date_col], date_format="%m/%y%")
train_df['group'] = 0
train_df = integer_index(train_df)

In [ ]:
from Training.Helper.dataPreprocessing import TRAIN_DATA_SPLIT
from sklearn.model_selection import train_test_split
import numpy as np

target_col = 'fred_PCEPI'

# input values are the integer indices, output values are PCEPI
train_X, val_X, train_y, val_y = train_test_split(list(train_df.index), train_df[target_col].values, train_size=TRAIN_DATA_SPLIT, shuffle=False)
train_X, val_X = np.array(train_X).reshape(-1, 1), np.array(val_X).reshape(-1, 1)

In [ ]:
from sklearn import linear_model

regr = linear_model.LinearRegression()
#linear model on just PCEPI
regr.fit(train_X, train_y)

In [ ]:
y_hat = regr.predict(val_X)

In [ ]:
y_train_hat = regr.predict(train_X)

In [ ]:
from matplotlib import pyplot as plt

y_hats = np.concatenate((y_train_hat, y_hat))

plt.figure(figsize=(20, 10))
ax = plt.axes()
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
plt.locator_params(axis='x', nbins=10)
plt.plot(train_df['observation_date'], y_hats)
plt.plot(train_df['observation_date'], train_df['fred_PCEPI'])
ax.fill_between(train_df['observation_date'][int(len(train_df)*TRAIN_DATA_SPLIT):], train_df['fred_PCEPI'][0], val_y[-1], facecolor='green', step='pre', alpha=0.5)
plt.title('Linear Regressor Predictions on PCEPI')
plt.xlabel('Date')
plt.ylabel('PCEPI')
plt.show()

In [ ]:
from Evaluation.Helper.evaluation_helpers import calc_metrics_arrays

# Metrics for normal linear regression model
calc_metrics_arrays(val_y, y_hat, model_names=['LR'])

### Test Set Predictions

In [ ]:
#re-'train' on full training set
regr.fit(np.concatenate((train_X, val_X)), np.concatenate((train_y, val_y)))

In [ ]:
final_val_index = val_X[-1][0]
print(f'Final date in validation set: {train_df.iloc[final_val_index]["observation_date"]}')

In [ ]:
from Training.Helper.dataPreprocessing import TEST_DATA_PATH_1990S, integer_index

date_col = 'observation_date'

# Load and format test data (only using PCEPI)
test_df = pd.read_csv(TEST_DATA_PATH_1990S, parse_dates=[date_col], date_format="%m/%y%").iloc[:,:2]
test_df = integer_index(test_df, start=final_val_index+1)

In [ ]:
test_X, test_y = np.array(list(test_df.index)).reshape(-1, 1), test_df[target_col]

In [ ]:
y_test_hat = regr.predict(test_X)
y_train_hat  = regr.predict(train_X)
y_hat = regr.predict(val_X)

In [ ]:
train_date_end = int(len(train_df)*TRAIN_DATA_SPLIT)
val_date_end = len(train_df)

In [ ]:
from matplotlib import pyplot as plt

# Make a plot showing all actual values and all predictions, including validation and test sets highlighted in different colours

y_hats = np.concatenate((y_train_hat, y_hat, y_test_hat))
ys = np.concatenate((train_y, val_y, test_y))
xs = np.concatenate((train_X, val_X, test_X))
df_dates = pd.concat((train_df[date_col], test_df[date_col]))

plt.figure(figsize=(10, 5))
ax = plt.axes()
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
plt.locator_params(axis='x', nbins=10)
plt.plot(df_dates, y_hats)
plt.plot(df_dates, ys)
ax.fill_between(df_dates[train_date_end:val_date_end+1], train_y[0], max(y_hats[-1], ys[-1]), facecolor='green', step='pre', alpha=0.5)
ax.fill_between(df_dates[val_date_end:], train_y[0], max(y_hats[-1], ys[-1]), facecolor='orange', step='pre', alpha=0.5)
plt.title('Linear Regressor Predictions on PCEPI (with test)')
plt.xlabel('Date')
plt.ylabel('PCEPI')
plt.legend(['Linear prediction', 'Actual PCEPI'])
plt.show()

### Saving Test Set Predictions

In [ ]:
train_exog = train_df.drop([target_col], axis=1)
train_exog[date_col] = train_exog.index  # Replace date strings with integer indices for the regressor to pick up on
train_exog = train_exog.values

In [ ]:
train_exog_X, val_exog_X = train_test_split(train_exog, train_size=TRAIN_DATA_SPLIT)

In [ ]:
mlr = linear_model.LinearRegression()
mlr.fit(train_exog_X, train_y)

In [ ]:
y_hat_val_exog = mlr.predict(val_exog_X)

In [ ]:
preds = regr.predict(test_X)
horizons = [1, 3, 6, 12]
predictions = [preds.copy() for _ in horizons]

In [ ]:
# MLR forecasts are all the same regardless of horizon, since does not adapt to input data
for i, horizon in enumerate(horizons):
    output_path = os.path.join(project_root, "Predictions", f"Horizon{horizon}", f"MLR_horizon_{horizon}.npy")
    np.save(output_path, predictions[i])

### Pytorch Forecasting Linear Regression Model

In [ ]:
import torch
from typing import Dict
from pytorch_forecasting.models import BaseModel
from pytorch_forecasting.metrics.point import RMSE

class LinearRegressionModule(torch.nn.Module):
 
    def __init__(self, input_size : int, output_size : int):
        super(LinearRegressionModule, self).__init__()
        #a single 1-1 linear function
        self.linear = torch.nn.Linear(input_size, output_size)
 
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred

In [ ]:
class LinearRegressionModel(BaseModel):
    def __init__(self, input_size: int, output_size: int, **kwargs):
        # saves arguments in signature to `.hparams` attribute, mandatory call - do not skip this
        self.save_hyperparameters()
        # pass additional arguments to BaseModel.__init__, mandatory call - do not skip this
        super().__init__(loss=RMSE(), **kwargs)
        self.network = LinearRegressionModule(
            input_size=self.hparams.input_size,
            output_size=self.hparams.output_size
        )

    def forward(self, x: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        # x is a batch generated based on the TimeSeriesDataset
        network_input = x["encoder_cont"].squeeze(-1)
        prediction = self.network(network_input)

        # rescale predictions into target space
        prediction = self.transform_output(prediction, target_scale=x["target_scale"])

        # We need to return a dictionary that at least contains the prediction
        # The parameter can be directly forwarded from the input.
        # The conversion to a named tuple can be directly achieved with the `to_network_output` function.
        return self.to_network_output(prediction=prediction)

In [ ]:
from pytorch_forecasting import TimeSeriesDataSet

encoderLength = 10
predictionLength = 10

trainDataset = TimeSeriesDataSet(
    train,
    group_ids=['group'],
    target='fred_PCEPI',
    time_idx='time_idx',
    min_encoder_length=encoderLength,
    max_encoder_length=encoderLength,
    min_prediction_length=predictionLength,
    max_prediction_length=predictionLength,
    time_varying_unknown_reals=['fred_PCEPI'],
)

valDataset = TimeSeriesDataSet.from_dataset(trainDataset, train_df, predict=True, stop_randomization=True)

In [ ]:
batch_size = 1
trainLoader = trainDataset.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
valLoader = valDataset.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

No categorical variables, one continuous variable, one continuous target variable. Normalise target values.

In [ ]:
model = LinearRegressionModel.from_dataset(trainDataset, input_size=encoderLength, output_size=predictionLength)

In [ ]:
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import EarlyStopping

early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")

#turned off logging, but by default this produces Tensorboard-interpretable logs every 50 steps
trainer = Trainer(fast_dev_run=False, callbacks=[early_stop_callback], logger=False)
trainer.fit(model, train_dataloaders=trainLoader, val_dataloaders=valLoader)

The above takes a long time to train a simple linear regressor.

In [ ]:
predictions = model.predict(
    valLoader, return_y=True, trainer_kwargs=dict(accelerator="cpu")
)
RMSE()(predictions.output, predictions.y)

In [ ]:
# raw predictions are a dictionary from which all kind of information including quantiles can be extracted
raw_predictions = model.predict(
    valLoader, mode="raw", return_x=True, trainer_kwargs=dict(accelerator="cpu")
)
print(raw_predictions.x)

In [ ]:
model.plot_prediction(
    raw_predictions.x, raw_predictions.output, idx=0, add_loss_to_title=True
)

Seem to have got unlucky here?